# Premier League Data Scraping

### Included in this file - 
1. A breakdown of how the data is extracted for each individual team
2. Reading the data for multiple teams, multiple seasons per team and exporting the data into a .csv 




## 1. Breakdown of how the data is extracted for the first team.

In [1]:
import requests

data_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


In [2]:
data = requests.get(data_url)

In [3]:
from bs4 import BeautifulSoup

# extract the table containing the team stats
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

In [4]:
# extract all links from the table
team_links = standings_table.find_all('a')

# extract all links from the a tags
all_links = [link.get('href') for link in team_links]

# extract only the squad links
squad_links = [link for link in all_links if '/en/squads/' in link]

In [5]:
# create full link
team_urls = [f"https://fbref.com{link}" for link in squad_links]

team_urls[0]


'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats'

In [6]:
import pandas as pd

# get team data for the first team (manchester city)
team_url = team_urls[0]

data = requests.get(team_url)


In [7]:
# convert to dataframe
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN


In [8]:
# scrape the shooting data for manchester city and store in a dataframe

soup = BeautifulSoup(data.text)
links = soup.find_all('a') # find all a tags
links = [link.get('href') for link in links] # extract the href attribute
links = [link for link in links if link and 'all_comps/shooting/' in link] # filter out the links that don't contain 'all_comps/shooting'

links


['/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [9]:
data = requests.get(f"https://fbref.com{links[0]}")

In [10]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [11]:
shooting.describe()

Standard                                                            \
              Gls           Sh         SoT       SoT%       G/Sh      G/SoT   
count   59.000000    59.000000   59.000000  59.000000  59.000000  59.000000   
mean     4.949153    35.661017   12.237288  33.955932   0.126949   0.372203   
std     18.772158   134.683871   46.281500  12.574211   0.089390   0.264608   
min      0.000000    10.000000    1.000000   6.300000   0.000000   0.000000   
25%      1.000000    15.000000    4.000000  25.000000   0.065000   0.170000   
50%      2.000000    18.000000    6.000000  33.300000   0.130000   0.370000   
75%      4.000000    21.500000    8.000000  40.600000   0.190000   0.500000   
max    146.000000  1052.000000  361.000000  61.500000   0.330000   1.000000   

                                                     Expected              \
            Dist         FK         PK      PKatt          xG        npxG   
count  51.000000  51.000000  59.000000  59.000000   51.000000   51.000000   
mean   15.845098   1.058824   0.372881   0.474576    4.437255    4.103922   
std     1.666531   3.770474   1.472721   1.850964   15.551938   14.384352   
min    11.600000   0.000000   0.000000   0.000000    0.700000    0.700000   
25%    14.750000   0.000000   0.000000   0.000000    1.600000    1.300000   
50%    15.700000   0.000000   0.000000   0.000000    2.100000    1.900000   
75%    16.850000   1.000000   0.000000   0.000000    3.050000    2.800000   
max    19.000000  27.000000  11.000000  14.000000  113.100000  104.600000   

                                        
         npxG/Sh       G-xG    np:G-xG  
count  51.000000  51.000000  51.000000  
mean    0.119216   0.856863   0.798039  
std     0.040192   4.780889   4.435200  
min     0.050000  -2.500000  -2.700000  
25%     0.090000  -0.600000  -0.600000  
50%     0.120000   0.100000   0.100000  
75%     0.140000   1.250000   1.250000  
max     0.220000  32.900000  30.400000

In [12]:
# drop the first level of indexing 
shooting.columns = shooting.columns.droplevel()

In [13]:
# merge the match information with the shooting data
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

team_data.describe()

,xG,xGA,Poss,Attendance,Sh,SoT,Dist,FK,PK,PKatt
count,50.000000,50.000000,58.000000,56.000000,58.000000,58.000000,50.00000,50.000000,58.000000,58.000000
mean,2.264000,0.774000,66.034483,45755.625000,18.137931,6.224138,15.84800,0.540000,0.189655,0.241379
std,1.004025,0.568137,8.048026,14188.773408,4.868258,2.967917,1.68332,0.705951,0.437573,0.470548
min,0.700000,0.000000,49.000000,13405.000000,10.000000,1.000000,11.60000,0.000000,0.000000,0.000000
25%,1.600000,0.325000,60.250000,34850.000000,15.000000,4.000000,14.72500,0.000000,0.000000,0.000000
50%,2.050000,0.750000,67.000000,52172.000000,17.500000,6.000000,15.70000,0.000000,0.000000,0.000000
75%,2.975000,1.000000,71.750000,53207.250000,21.000000,8.000000,16.87500,1.000000,0.000000,0.000000
max,4.600000,2.600000,80.000000,73793.000000,31.000000,15.000000,19.00000,3.000000,2.000000,2.000000


## 2. How the data is extracted for multiple teams for multiple seasons

In [14]:
years = list(range(2022, 2019, -1))
years

[2022, 2021, 2020]

In [15]:
all_matches = []

In [16]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [17]:
import time
for year in years:

    # get the premier league table for the current year
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    # each team has a link in the table, find all the team links
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # on the page there is a button to go the previous season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    # get the data for each team
    for team_url in team_urls:

        #clean the team name
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        # get the data for the current team page
        data = requests.get(team_url)

        # the season data is stored in a "Scores & Fixtures" table
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)

        # on each page there is a button to get the shooting data for each match
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")

        # the shooting data is stored in a "Shooting" table
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        # merge the match information with the shooting data
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        # filter by competition
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        # add in season and team columns
        team_data["Season"] = year
        team_data["Team"] = team_name

        # add data to all_matches and sleep for a bit to avoid being blocked
        all_matches.append(team_data)
        time.sleep(0.5)
    

    

            


In [ ]:
# merge all the data together and lowercase the column names
match_data = pd.concat(all_matches)
match_data.columns = [c.lower() for c in match_data.columns]

#write data to a csv file
match_data.to_csv("premier_league_data.csv")
